# Đọc dữ liệu

In [0]:
test_df = spark.table("workspace.default.customer_churn_dataset_testing_master")
train_df  = spark.table("workspace.default.customer_churn_dataset_training_master")

# Thống kê

In [0]:
print("Train rows:", train_df.count())
print("Test rows:", test_df.count())

In [0]:
train_df.printSchema()

In [0]:
from pyspark.sql.functions import col

train_df.groupBy("Churn").count().show()

total = train_df.count()

train_df.groupBy("Churn") \
    .count() \
    .withColumn("percentage", col("count")/total * 100) \
    .show()

In [0]:
display(train_df.describe())

In [0]:
display(
    train_df.groupBy("Churn").count()
)

Databricks visualization. Run in Databricks to view.

- Dataset không bị mất cân bằng nghiêm trọng, nhưng số lượng khách hàng churn nhiều hơn khách hàng không churn.
- Tỷ lệ churn khá cao → cho thấy đây là bài toán quan trọng.
- Không cần xử lý imbalance mạnh (như SMOTE) ở bước đầu.
>Tỷ lệ churn chiếm phần lớn trong tập dữ liệu, cho thấy hành vi rời bỏ dịch vụ là phổ biến và cần được phân tích sâu hơn.

In [0]:
display(
    train_df.select("Age")
)

Databricks visualization. Run in Databricks to view.

* Dataset có độ tuổi trải đều, không thiên lệch về một nhóm tuổi.
* Không cần xử lý outlier ở biến Age.
* Có thể tuổi không phải yếu tố quyết định mạnh.
> Độ tuổi khách hàng phân bố tương đối đồng đều, không xuất hiện giá trị bất thường.

In [0]:
display(
    train_df.groupBy("Churn")
            .avg("Total Spend")
)

Databricks visualization. Run in Databricks to view.

- Khách hàng chi tiêu thấp có xu hướng churn nhiều hơn.
- Total Spend là feature rất tiềm năng.
- Có thể khách hàng giá trị thấp ít gắn bó.
> Khách hàng có mức chi tiêu trung bình thấp hơn có xu hướng rời bỏ dịch vụ nhiều hơn, cho thấy Total Spend là biến có ảnh hưởng đáng kể đến churn.

In [0]:
display(
    train_df.groupBy("Churn")
            .avg("Support Calls")
)

Databricks visualization. Run in Databricks to view.

- Khách hàng gọi hỗ trợ nhiều có xác suất churn cao.
- Đây là dấu hiệu không hài lòng.
- Support Calls có thể là feature dự báo rất mạnh.
>Số lần liên hệ hỗ trợ của nhóm khách hàng churn cao gấp nhiều lần nhóm không churn, cho thấy mức độ không hài lòng có liên hệ chặt chẽ với hành vi rời bỏ.

# Tiền xử lí

## Drop Null

In [0]:
train_df = train_df.dropna()
test_df  = test_df.dropna()

## Drop chuỗi rỗng

In [0]:
from pyspark.sql.functions import col, trim

string_cols = [c for (c, t) in train_df.dtypes if t == "string"]

for c in string_cols:
    train_df = train_df.filter(trim(col(c)) != "")
    test_df  = test_df.filter(trim(col(c)) != "")

## Ép kiểu label Churn về kiểu int

In [0]:
train_df = train_df.withColumn("Churn", col("Churn").cast("int"))
test_df  = test_df.withColumn("Churn", col("Churn").cast("int"))

In [0]:
label_col = "Churn"

categorical_cols = [
    c for (c, t) in train_df.dtypes
    if t == "string" and c not in ["CustomerID"]
]

numeric_cols = [
    c for (c, t) in train_df.dtypes
    if t in ["int", "double", "float", "bigint"]
    and c not in ["Churn"]
]

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

indexers = [
    StringIndexer(
        inputCol=c,
        outputCol=c + "_index",
        handleInvalid="keep"
    )
    for c in categorical_cols
]

encoder = OneHotEncoder(
    inputCols=[c + "_index" for c in categorical_cols],
    outputCols=[c + "_vec" for c in categorical_cols]
)

pipeline = Pipeline(stages=indexers + [encoder])

preprocess_model = pipeline.fit(train_df)

In [0]:
train_processed = preprocess_model.transform(train_df)
test_processed  = preprocess_model.transform(test_df)

In [0]:
from pyspark.ml.feature import VectorAssembler

feature_cols = numeric_cols + [c + "_vec" for c in categorical_cols]

assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features"
)

train_processed = assembler.transform(train_processed)
test_processed  = assembler.transform(test_processed)

In [0]:
train_processed.select("features", "Churn").show(5, truncate=False)
test_processed.select("features", "Churn").show(5, truncate=False)